In [19]:
#Extracting PM2.5 levels from the CPCB data
import pandas as pd

# Load the CSV
pm_df = pd.read_csv("CPCB Data/City_wise_raw_data_1Hr_2019_Faridabad_1Hr.csv")

# Parse Timestamp column
pm_df['Timestamp'] = pd.to_datetime(pm_df['Timestamp'], errors='coerce')

# Create a date range for 1st to 18th June
date_range = pd.date_range(start='2019-01-01 05:00:00', end='2019-12-31 05:00:00', freq='D')

# Filter for only 05:00 data in that date range
pm_filtered = pm_df[pm_df['Timestamp'].isin(date_range)]

# Keep only Timestamp and PM2.5 columns
pm_filtered = pm_filtered[['Timestamp', 'PM2.5 (µg/m³)']]

# Optional: Sort by date
pm_filtered = pm_filtered.sort_values('Timestamp')

# Print the results
print(pm_filtered)

# Save for training
pm_filtered.to_csv("pm25_cpcb_05AM_01janto31dec_2019.csv", index=False)


               Timestamp  PM2.5 (µg/m³)
5    2019-01-01 05:00:00         260.00
29   2019-01-02 05:00:00         283.00
53   2019-01-03 05:00:00         298.00
77   2019-01-04 05:00:00         230.00
101  2019-01-05 05:00:00         295.00
...                  ...            ...
8645 2019-12-27 05:00:00         193.00
8669 2019-12-28 05:00:00         307.50
8693 2019-12-29 05:00:00         253.75
8717 2019-12-30 05:00:00         306.50
8741 2019-12-31 05:00:00         137.50

[365 rows x 2 columns]


In [1]:
#Combining the two csv files
import pandas as pd
df1 = pd.read_csv("combined_pm2.5_para.csv")
df2 = pd.read_csv("aod_data.csv")
combined_df = pd.concat([df1,df2], ignore_index= True)
combined_df.to_csv("combined_pm2.5_para.csv", index=False)
print("Files combined successfully!")

Files combined successfully!


In [9]:
#Trying to identify the shape of AOD, Latitude, and Longitude
import h5py

file_path = "AOD Data/3DIMG_15JUN2024_0530_L2G_AOD_V02R00.h5"

with h5py.File(file_path, 'r') as f:
    print("Full structure of the HDF5 file:")
    def print_structure(name, obj):
        if isinstance(obj, h5py.Dataset):
            print(f"Dataset: {name} — shape: {obj.shape}")
    f.visititems(print_structure)


Full structure of the HDF5 file:
Dataset: AOD — shape: (1, 551, 551)
Dataset: latitude — shape: (551,)
Dataset: longitude — shape: (551,)
Dataset: time — shape: (1,)


In [10]:
import h5py
import numpy as np
import os
import pandas as pd

# Folder where your HDF5 files are stored
folder_path = "AOD Data/"

# Define Faridabad region bounds
lat_min, lat_max = 28.2, 28.5
lon_min, lon_max = 77.2, 77.5

# Initialize list to store results
results = []

# List all .h5 files in the folder
for filename in sorted(os.listdir(folder_path)):
    if filename.endswith(".h5"):
        file_path = os.path.join(folder_path, filename)
        
        try:
            with h5py.File(file_path, 'r') as f:
                # Load datasets
                aod = f['AOD'][:].squeeze()
                lat = f['latitude'][:]
                lon = f['longitude'][:]
                
                # Flip for correct orientation
                lat_flipped = lat[::-1]
                aod_flipped = aod[::-1, :]
                lon2d, lat2d = np.meshgrid(lon, lat_flipped)

                # Apply mask for Faridabad region
                mask = (lat2d >= lat_min) & (lat2d <= lat_max) & (lon2d >= lon_min) & (lon2d <= lon_max)
                aod_faridabad_values = aod_flipped[mask]
                mean_aod = np.nanmean(aod_faridabad_values)

                # Extract date from filename (assumes format: 3DIMG_15JUN2024_0530_....h5)
                date_str = filename.split("_")[1]
                date = pd.to_datetime(date_str, format='%d%b%Y')

                # Append to results
                results.append({'Date': date, 'Mean_AOD': mean_aod})

        except Exception as e:
            print(f"Failed to process {filename}: {e}")

# Convert results to DataFrame
df = pd.DataFrame(results)
df.sort_values('Date', inplace=True)

# Save to CSV for training
df.to_csv("mean_aod_faridabad.csv", index=False)

print(df)


         Date    Mean_AOD
0  2024-06-01 -999.000000
1  2024-06-02 -776.836670
2  2024-06-03 -776.832703
3  2024-06-04 -999.000000
4  2024-06-05 -776.539917
5  2024-06-06 -776.747070
6  2024-06-07 -776.795715
7  2024-06-08 -999.000000
8  2024-06-09 -776.817932
9  2024-06-10 -776.841675
10 2024-06-11 -887.868042
11 2024-06-12 -776.887695
12 2024-06-13 -776.807800
13 2024-06-14 -776.628784
14 2024-06-15 -776.783203
15 2024-06-16 -776.845398
16 2024-06-17 -776.820801
17 2024-06-18 -776.784424


In [11]:
#Combine AOD and PM2.5 into One Dataframe
import pandas as pd

# Load both datasets
aod_df = pd.read_csv("mean_aod_faridabad.csv")
pm_df = pd.read_csv("pm25_cpcb_05AM_1to18june.csv")

# Convert date columns to datetime
aod_df['Date'] = pd.to_datetime(aod_df['Date'])
pm_df['Timestamp'] = pd.to_datetime(pm_df['Timestamp'])

# Extract just the date part but KEEP datetime64[ns] type
pm_df['Date'] = pm_df['Timestamp'].dt.normalize()

# Merge on Date
combined_df = pd.merge(aod_df, pm_df, left_on='Date', right_on='Date')

# Rename for simplicity
combined_df.rename(columns={"PM2.5 (µg/m³)": "PM2.5"}, inplace=True)

print(combined_df.head())


        Date  Mean_AOD           Timestamp  PM2.5
0 2024-06-01 -999.0000 2024-06-01 05:00:00  76.40
1 2024-06-02 -776.8367 2024-06-02 05:00:00  74.98
2 2024-06-03 -776.8327 2024-06-03 05:00:00  77.86
3 2024-06-04 -999.0000 2024-06-04 05:00:00  55.40
4 2024-06-05 -776.5399 2024-06-05 05:00:00  88.99


In [12]:
#Training a simple linear regression model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Features and target
X = combined_df[['Mean_AOD']]  # Feature
y = combined_df['PM2.5']       # Target

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Mean Absolute Error on test set:", mean_absolute_error(y_test, y_pred))

#Predicting PM2.5 for 19th June
aod_19_june = 0.97  # Replace this with your actual computed value
predicted_pm = model.predict([[aod_19_june]])
print("Predicted PM2.5 for 2024-06-19 at 05:00:", predicted_pm[0])


Mean Absolute Error on test set: 16.329789183384506
Predicted PM2.5 for 2024-06-19 at 05:00: 59.31425022570035


c:\Users\91931\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [14]:
#Prediction by Random Forest
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Load and merge the datasets
aod_df = pd.read_csv("mean_aod_faridabad.csv")
pm_df = pd.read_csv("pm25_cpcb_05AM_1to18june.csv")

# Convert to datetime
aod_df['Date'] = pd.to_datetime(aod_df['Date'])
pm_df['Timestamp'] = pd.to_datetime(pm_df['Timestamp'])
pm_df['Date'] = pm_df['Timestamp'].dt.normalize()

# Merge on 'Date'
combined_df = pd.merge(aod_df, pm_df, on='Date')
combined_df.rename(columns={"PM2.5 (µg/m³)": "PM2.5"}, inplace=True)

# Select features and target
X = combined_df[['Mean_AOD']]
y = combined_df['PM2.5']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate model
y_pred = rf.predict(X_test)

print("✅ Model Evaluation:")
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R² Score:", r2_score(y_test, y_pred))

#Prediction for 19th June
aod_19_june = 0.97

predicted_pm25 = rf.predict([[aod_19_june]])
print("\n🔮 Predicted PM2.5 for 19 June 2024 at 05:00:", predicted_pm25[0])


✅ Model Evaluation:
MAE: 7.778910714285711
RMSE: 8.834053260522328
R² Score: 0.5988373888613501

🔮 Predicted PM2.5 for 19 June 2024 at 05:00: 77.30559999999991


c:\Users\91931\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [ ]:
#Combining MOSDAC and CPCB data with MERRA and predicting the PM2.5 Level
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from netCDF4 import Dataset, num2date
from datetime import datetime, timedelta
import os
import h5py

# ---------- Step 1: Load INSAT AOD from CSV ----------
aod_df = pd.read_csv("aod_data.csv")
aod_df['Date'] = pd.to_datetime(aod_df['Date'])  # 🔄 Convert to datetime for merging
# print(aod_df.head())

# ---------- Step 2: Load CPCB PM2.5 ----------
pm_df = pd.read_csv("combined_pm2.5_para.csv")
pm_df['Timestamp'] = pd.to_datetime(pm_df['Timestamp'])
pm_df['Date'] = pm_df['Timestamp'].dt.normalize()

# ---------- Step 3: Load MERRA .nc4 files ----------
def extract_merra_features(nc_folder):
    records = []
    for file in os.listdir(nc_folder):
        if file.endswith(".nc"):
            ds = Dataset(os.path.join(nc_folder, file), 'r')

            time_var = ds.variables['time']
            times = num2date(time_var[:], units=time_var.units, only_use_cftime_datetimes=False)

            ps = ds.variables['PS'][:, 0, 0]
            qv2m = ds.variables['QV2M'][:, 0, 0]
            t2m = ds.variables['T2M'][:, 0, 0]
            ts = ds.variables['TS'][:, 0, 0]
            u10m = ds.variables['U10M'][:, 0, 0]
            # v10m = ds.variables['V10M'][:, 0, 0]
            qv10m = ds.variables['QV10M'][:, 0, 0]
            slp = ds.variables['SLP'][:, 0, 0]
            t10m = ds.variables['T10M'][:, 0, 0]
            t2mdew = ds.variables['T2MDEW'][:, 0, 0]
            tqi = ds.variables['TQI'][:, 0, 0]
            tql = ds.variables['TQL'][:, 0, 0]

            for i in range(len(times)):
                # Ensure times[i] is a native datetime object
                date_val = times[i]
                if hasattr(date_val, 'year'):
                    date_val = datetime(date_val.year, date_val.month, date_val.day)

                records.append({
                    "Date": date_val,
                    "PS": ps[i],
                    "QV2M": qv2m[i],
                    "T2M": t2m[i],
                    "TS": ts[i],
                    "U10M": u10m[i],
                    "QV10M": qv10m[i],
                    "SLP": slp[i],
                    "T10M": t10m[i],
                    "T2MDEW": t2mdew[i],
                    "TQI": tqi[i],
                    "TQL": tql[i],
                    # "U2M": u2m[i]
                })

    return pd.DataFrame(records)

merra_df = extract_merra_features("merra_downloads")  # 📁 Folder containing .nc4 files
merra_df = merra_df.groupby("Date").mean().reset_index()

aod_df['Date'] = pd.to_datetime(aod_df['Date'])
pm_df['Date'] = pd.to_datetime(pm_df['Date'])
merra_df['Date'] = pd.to_datetime(merra_df['Date'])  # ✅ This fixes the issue

# print(aod_df.dtypes)
# print(pm_df.dtypes)
# print(merra_df.dtypes)

# print(f"AOD records: {len(aod_df)}")
# print(f"CPCB records: {len(pm_df)}")
# print(f"MERRA records: {len(merra_df)}")

# ---------- Step 4: Merge All ----------
combined_df = pd.merge(aod_df, pm_df, on="Date")
# print(f"After merging AOD + CPCB: {len(combined_df)} records")

combined_df = pd.merge(combined_df, merra_df, on="Date")
# print(f"After merging with MERRA: {len(combined_df)} records")

# print("🔍 AOD dates:", aod_df['Date'].unique())
# print("🔍 CPCB dates:", pm_df['Date'].unique())
# print("🔍 MERRA dates:", merra_df['Date'].unique())


# ---------- Step 5: ML Model ----------
features = ['Mean_AOD', 'PS', 'QV2M', 'T2M', 'TS', 'U10M', 'QV10M', 'SLP', 'T10M', 'T2MDEW', 'TQI', 'TQL']
clean_df = combined_df.dropna(subset=features + ['PM2.5 (µg/m³)'])

# Create feature matrix and target vector
X = clean_df[features].copy()
y = clean_df['PM2.5 (µg/m³)']

# Optional feature engineering
X['Temp_Diff'] = X['TS'] - X['T2M']
X['Humidity_Ratio'] = X['QV2M'] / (X['T2M'] + 1e-3)  # prevent division by zero

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("✅ Evaluation:")
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R²:", r2_score(y_test, y_pred))

# # ---------- Step 6: Predict for 20 June ----------
# # Replace with actual June 20 values from your MERRA and AOD
may_11 = {
    "Mean_AOD": [0.97],
    "PS": [98119.390],
    "QV2M": [0.007],
    "T2M": [313.68],
    "TS": [316.82],
    "U10M": [2.75],
    "QV10M": [0.007],
    "SLP": [100396.60],
    "T10M": [312.85],
    "T2MDEW": [283.28],
    "TQI": [0.0],
    "TQL": [0.0],
}
may_11_df = pd.DataFrame(may_11)

# Recreate engineered features
may_11_df['Temp_Diff'] = may_11_df['TS'] - may_11_df['T2M']
may_11_df['Humidity_Ratio'] = may_11_df['QV2M'] / (may_11_df['T2M'] + 1e-3)

# Ensure columns match training
X_input = may_11_df[X_train.columns]

# Predict
pred_pm = rf.predict(X_input)
print("\n🔮 Predicted PM2.5 for 11 May 2024 at 05:30 IST:", pred_pm[0])

# ---------- Step 7: Save Prediction to CSV ----------
output_df = pd.DataFrame({
    'Date': ['2024-05-11'],
    'Predicted_PM2.5': [pred_pm[0]]
})

# Save to CSV (overwrite every time)
output_df.to_csv('predicted_pm25.csv', index=False, mode='w')

print("💾 Prediction saved to 'predicted_pm25.csv'")



✅ Evaluation:
MAE: 28.026991176470588
RMSE: 36.08490578960144
R²: 0.0041481018634580424

🔮 Predicted PM2.5 for 11 May 2024 at 05:30 IST: 96.84259999999999
💾 Prediction saved to 'predicted_pm25.csv'


In [ ]:
#To read AOD value from a single .h5 file
import h5py
import numpy as np
from datetime import datetime
import os

# 📍 Location of interest: Faridabad (approx.)
target_lat = 28.4
target_lon = 77.3

# 📄 Path to one .h5 file (example)
file_path = "aod_folder/3DIMG_01MAY2024_0530_L2G_AOD_V02R00.h5"

# ✅ Read and extract AOD value
with h5py.File(file_path, 'r') as f:
    print("🔍 Keys in file:", list(f.keys()))  # Show root-level keys

    # Check if required datasets are present
    if 'AOD' in f and 'latitude' in f and 'longitude' in f:
        aod_data = f['AOD'][:]           # 2D array of AOD
        latitudes = f['latitude'][:]     # 1D array
        longitudes = f['longitude'][:]   # 1D array

        print("📐 AOD shape:", aod_data.shape)
        print("🧭 Latitude shape:", latitudes.shape)
        print("🧭 Longitude shape:", longitudes.shape)

        # Find nearest grid index to target lat/lon
        lat_idx = (np.abs(latitudes - target_lat)).argmin()
        lon_idx = (np.abs(longitudes - target_lon)).argmin()
        print("🔎 Nearest index — lat:", lat_idx, "lon:", lon_idx)

        # Handle possible shape issues
        try:
            aod_value = aod_data[0, lat_idx, lon_idx]
            print("📌 AOD value at Faridabad:", aod_value)
        except IndexError as e:
            print("❌ IndexError while accessing AOD value:", e)
            aod_value = np.nan
    else:
        print("❌ One or more required datasets not found in file.")
        aod_value = np.nan

    # Parse date from filename
    filename = os.path.basename(file_path)
    try:
        date_str = filename.split("_")[1]  # '01JUN2024'
        date_obj = datetime.strptime(date_str, "%d%b%Y").date()
        print("📆 Date parsed from filename:", date_obj)
    except Exception as e:
        print("❌ Error parsing date from filename:", e)
        date_obj = None

    # Store result
    record = {
        "Date": date_obj,
        "Mean_AOD": aod_value
    }

print("✅ Final Record:", record)


🔍 Keys in file: ['AOD', 'latitude', 'longitude', 'time']
📐 AOD shape: (1, 551, 551)
🧭 Latitude shape: (551,)
🧭 Longitude shape: (551,)
🔎 Nearest index — lat: 166 lon: 322
📌 AOD value at Faridabad: 0.779472
📆 Date parsed from filename: 2024-05-01
✅ Final Record: {'Date': datetime.date(2024, 5, 1), 'Mean_AOD': 0.779472}


In [2]:
#Merging all the .h5 into a single csv file
import h5py
import numpy as np
import pandas as pd
from datetime import datetime
import os

def extract_aod_from_folder(folder_path):
    records = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".h5"):
            file_path = os.path.join(folder_path, filename)
            try:
                with h5py.File(file_path, 'r') as f:
                    # Ensure keys exist (some files might be corrupt)
                    if 'AOD' not in f or 'latitude' not in f or 'longitude' not in f:
                        continue

                    aod_data = f['AOD'][:]
                    latitudes = f['latitude'][:]
                    longitudes = f['longitude'][:]

                    # Find nearest point to Faridabad
                    lat_idx = (np.abs(latitudes - 28.4)).argmin()
                    lon_idx = (np.abs(longitudes - 77.3)).argmin()

                    # Fix for shape mismatch
                    if aod_data.ndim == 3 and lat_idx < aod_data.shape[1] and lon_idx < aod_data.shape[2]:
                        aod_val = aod_data[0, lat_idx, lon_idx]
                    else:
                        aod_val = np.nan  # skip if shape mismatch

                    # Extract date from filename
                    date_str = filename.split("_")[1]  # '01JUN2024'
                    date_obj = datetime.strptime(date_str, "%d%b%Y").date()

                    records.append({
                        "Date": date_obj,
                        "Mean_AOD": aod_val
                    })

            except Exception as e:
                print(f"❌ Failed for {filename}: {e}")
    
    return pd.DataFrame(records)

# 🔁 Extract from all files
aod_df = extract_aod_from_folder("aod_folder")

# 💾 Save to CSV
aod_df.to_csv("aod_data.csv", index=False)
print("✅ Saved all AOD values to aod_data.csv")


✅ Saved all AOD values to aod_data.csv


In [39]:
#Reading the MERRA Varible values from the file
import xarray as xr

# Load NetCDF file
file_path = "merra_downloads\MERRA2_400.tavg1_2d_slv_Nx.20240511.SUB.nc"
ds = xr.open_dataset(file_path)

# 🔍 Step 1: Define coordinates of Faridabad (approx)
target_lat = 28.41
target_lon = 77.31

# 🔍 Step 2: Select the nearest location
nearest_lat = ds.sel(lat=target_lat, method="nearest").lat.values
nearest_lon = ds.sel(lon=target_lon, method="nearest").lon.values

# 🔍 Step 3: Select the first time step (if only one day is present)
time_step = ds.time.values[0]

# 🔍 Step 4: Extract values for each variable at that location and time
may_11 = {
    "PS":     [float(ds["PS"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
    "QV2M":   [float(ds["QV2M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
    "T2M":    [float(ds["T2M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
    "TS":     [float(ds["TS"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
    "U10M":   [float(ds["U10M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
    "QV10M":   [float(ds["QV10M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
    "SLP":   [float(ds["SLP"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
    "T10M":   [float(ds["T10M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
    "T2MDEW":   [float(ds["T2MDEW"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
    "TQI":   [float(ds["TQI"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
    "TQL":   [float(ds["TQL"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
    # "U2M":   [float(ds["U2M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
    # "Mean_AOD": [0.97]  # Add manually or fetch from INSAT/CPCB if needed
}

print(may_11)

{'PS': [98119.390625], 'QV2M': [0.007875049486756325], 'T2M': [313.6807861328125], 'TS': [316.8280334472656], 'U10M': [2.7565226554870605], 'QV10M': [0.007863740436732769], 'SLP': [100396.609375], 'T10M': [312.8543701171875], 'T2MDEW': [283.28582763671875], 'TQI': [0.0], 'TQL': [0.0]}


In [9]:
#Combining MOSDAC and CPCB data with MERRA and predicting the PM2.5 Level(SAutomatic filling of MERRA DATA)
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from netCDF4 import Dataset, num2date
from datetime import datetime
import os
import xarray as xr

# ---------- Step 1: Load INSAT AOD from CSV ----------
aod_df = pd.read_csv("aod_data.csv")

# Parse with mixed formats; keep rows that fail as NaT so you can inspect
aod_df['Date'] = pd.to_datetime(
    aod_df['Date'], 
    format='mixed',     # pandas >= 2.0
    dayfirst=True, 
    errors='coerce'
)

bad_aod = aod_df[aod_df['Date'].isna()]
if not bad_aod.empty:
    print("Unparsable AOD dates (fix in CSV):")
    print(bad_aod.head(10))

# ---------- Step 2: Load CPCB PM2.5 ----------
pm_df = pd.read_csv("combined_pm2.5_para.csv")

pm_df['Timestamp'] = pd.to_datetime(
    pm_df['Timestamp'], 
    format='mixed', 
    dayfirst=True, 
    errors='coerce'
)

bad_pm = pm_df[pm_df['Timestamp'].isna()]
if not bad_pm.empty:
    print("Unparsable CPCB timestamps (fix in CSV):")
    print(bad_pm[['Timestamp']].head(10))

pm_df['Date'] = pm_df['Timestamp'].dt.normalize()

# ---------- Step 3: Load MERRA .nc4 files ----------
def extract_merra_features(nc_folder):
    records = []
    for file in os.listdir(nc_folder):
        if file.endswith(".nc"):
            ds = Dataset(os.path.join(nc_folder, file), 'r')

            time_var = ds.variables['time']
            times = num2date(time_var[:], units=time_var.units, only_use_cftime_datetimes=False)

            ps = ds.variables['PS'][:, 0, 0]
            qv2m = ds.variables['QV2M'][:, 0, 0]
            t2m = ds.variables['T2M'][:, 0, 0]
            ts = ds.variables['TS'][:, 0, 0]
            u10m = ds.variables['U10M'][:, 0, 0]
            # v10m = ds.variables['V10M'][:, 0, 0]
            qv10m = ds.variables['QV10M'][:, 0, 0]
            slp = ds.variables['SLP'][:, 0, 0]
            t10m = ds.variables['T10M'][:, 0, 0]
            t2mdew = ds.variables['T2MDEW'][:, 0, 0]
            tqi = ds.variables['TQI'][:, 0, 0]
            tql = ds.variables['TQL'][:, 0, 0]

            for i in range(len(times)):
                # Ensure times[i] is a native datetime object
                date_val = times[i]
                if hasattr(date_val, 'year'):
                    date_val = datetime(date_val.year, date_val.month, date_val.day)

                records.append({
                    "Date": date_val,
                    "PS": ps[i],
                    "QV2M": qv2m[i],
                    "T2M": t2m[i],
                    "TS": ts[i],
                    "U10M": u10m[i],
                    "QV10M": qv10m[i],
                    "SLP": slp[i],
                    "T10M": t10m[i],
                    "T2MDEW": t2mdew[i],
                    "TQI": tqi[i],
                    "TQL": tql[i],
                    # "U2M": u2m[i]
                })

    return pd.DataFrame(records)

merra_df = extract_merra_features("merra_downloads")  
merra_df = merra_df.groupby("Date").mean().reset_index()

aod_df['Date'] = pd.to_datetime(aod_df['Date'])
pm_df['Date'] = pd.to_datetime(pm_df['Date'])
merra_df['Date'] = pd.to_datetime(merra_df['Date'])  

# print(aod_df.dtypes)
# print(pm_df.dtypes)
# print(merra_df.dtypes)

# print(f"AOD records: {len(aod_df)}")
# print(f"CPCB records: {len(pm_df)}")
# print(f"MERRA records: {len(merra_df)}")

# ---------- Step 4: Merge All ----------
combined_df = pd.merge(aod_df, pm_df, on="Date")
# print(f"After merging AOD + CPCB: {len(combined_df)} records")

combined_df = pd.merge(combined_df, merra_df, on="Date")
# print(f"After merging with MERRA: {len(combined_df)} records")

# print("🔍 AOD dates:", aod_df['Date'].unique())
# print("🔍 CPCB dates:", pm_df['Date'].unique())
# print("🔍 MERRA dates:", merra_df['Date'].unique())


# ---------- Step 5: ML Model ----------
features = ['Mean_AOD', 'PS', 'QV2M', 'T2M', 'TS', 'U10M', 'QV10M', 'SLP', 'T10M', 'T2MDEW', 'TQI', 'TQL']
clean_df = combined_df.dropna(subset=features + ['PM2.5 (µg/m³)'])

# Create feature matrix and target vector
X = clean_df[features].copy()
y = clean_df['PM2.5 (µg/m³)']

# Optional feature engineering
X['Temp_Diff'] = X['TS'] - X['T2M']
X['Humidity_Ratio'] = X['QV2M'] / (X['T2M'] + 1e-3)  # prevent division by zero

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("✅ Evaluation:")
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R²:", r2_score(y_test, y_pred))


# ---------- Step 6: Extract MERRA features for a specific date ----------
def extract_merra_single_day(file_path, lat=28.41, lon=77.31):
    ds = xr.open_dataset(file_path)

    nearest_lat = ds.sel(lat=lat, method="nearest").lat.values
    nearest_lon = ds.sel(lon=lon, method="nearest").lon.values

    time_step = ds.time.values[0]  # Assuming only one day present

    features = {
        "PS":     [float(ds["PS"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "QV2M":   [float(ds["QV2M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "T2M":    [float(ds["T2M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "TS":     [float(ds["TS"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "U10M":   [float(ds["U10M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "QV10M":  [float(ds["QV10M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "SLP":    [float(ds["SLP"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "T10M":   [float(ds["T10M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "T2MDEW": [float(ds["T2MDEW"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "TQI":    [float(ds["TQI"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "TQL":    [float(ds["TQL"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
    }

    return features

# Update this path to the correct file for the target date
merra_file = "merra_downloads/MERRA2_400.tavg1_2d_slv_Nx.20240403.SUB.nc"
merra_features = extract_merra_single_day(merra_file)

# Add manually or automate fetching AOD from INSAT later
merra_features["Mean_AOD"] = [0.97]

# Create DataFrame from extracted features
predict_input_df = pd.DataFrame(merra_features)

# ---------- Step 7: Feature Engineering ----------
predict_input_df['Temp_Diff'] = predict_input_df['TS'] - predict_input_df['T2M']
predict_input_df['Humidity_Ratio'] = predict_input_df['QV2M'] / (predict_input_df['T2M'] + 1e-3)

# Ensure correct column order
X_input = predict_input_df[X_train.columns]

# ---------- Step 8: Predict ----------
pred_pm = rf.predict(X_input)
print("\n🔮 Predicted PM2.5 for 03-04-2024 around 05:30 IST:", pred_pm[0])

# ---------- Step 9: Save prediction ----------
output_df = pd.DataFrame({
    'Date': ['2024-05-11'],
    'Predicted_PM2.5': [pred_pm[0]]
})
output_df.to_csv('predicted_pm25.csv', index=False, mode='w')
print("💾 Prediction saved to 'predicted_pm25.csv'")


✅ Evaluation:
MAE: 28.026991176470588
RMSE: 36.08490578960144
R²: 0.0041481018634580424

🔮 Predicted PM2.5 for 03-04-2024 around 05:30 IST: 105.40529999999995
💾 Prediction saved to 'predicted_pm25.csv'


In [8]:
#Finding PM2.5 concentration for a single day
import pandas as pd
df = pd.read_csv("CPCB Data/City_wise_raw_data_1Hr_2024_Faridabad_1Hr.csv")
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors = "coerce")
target_time = pd.Timestamp("2024-04-03 05:00:00")
pm_at_time = df[df["Timestamp"] == target_time]
print(pm_at_time[["Timestamp", "PM2.5 (µg/m³)"]])

               Timestamp  PM2.5 (µg/m³)
2237 2024-04-03 05:00:00         176.48


In [31]:
#Improved version of above acc to GPT-5
# -------------------- IMPORTS --------------------
import pandas as pd
import numpy as np
import os
import xarray as xr
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import datetime

# -------------------- CONSTANTS --------------------
CITY_LAT, CITY_LON = 28.41, 77.31   # Faridabad location
IST = "Asia/Kolkata"
MERRA_VARS = ["PS","QV2M","T2M","TS","U10M","QV10M","SLP","T10M","T2MDEW","TQI","TQL"]  # Add V10M if available

# -------------------- STEP 1: LOAD AOD --------------------
aod_df = pd.read_csv("aod_data.csv")
aod_df["Date"] = pd.to_datetime(aod_df["Date"])
aod_df["Date"] = aod_df["Date"].dt.tz_localize(IST, nonexistent="shift_forward", ambiguous="NaT").dt.floor("D").dt.tz_localize(None)

# -------------------- STEP 2: LOAD CPCB PM2.5 --------------------
pm_df = pd.read_csv("combined_pm2.5_para.csv")
pm_df["Timestamp"] = pd.to_datetime(pm_df["Timestamp"])
pm_df["Timestamp"] = pm_df["Timestamp"].dt.tz_localize(IST, nonexistent="shift_forward", ambiguous="NaT")
pm_df["Date"] = pm_df["Timestamp"].dt.floor("D").dt.tz_localize(None)

# Keep only date + PM columns
pm_df = pm_df[["Date", "PM2.5 (µg/m³)"]]

# -------------------- STEP 3: LOAD MERRA DATA --------------------
def extract_merra_features_xr(nc_folder, lat=CITY_LAT, lon=CITY_LON):
    frames = []
    for f in os.listdir(nc_folder):
        if f.endswith((".nc", ".nc4")):
            ds = xr.open_dataset(os.path.join(nc_folder, f))
            sub = ds.sel(lat=lat, lon=lon, method="nearest")[MERRA_VARS]
            df = sub.to_dataframe().reset_index()
            df["Date"] = (pd.to_datetime(df["time"])
                          .dt.tz_localize("UTC")
                          .dt.tz_convert(IST)
                          .dt.floor("D")
                          .dt.tz_localize(None))
            df = df.groupby("Date")[MERRA_VARS].mean().reset_index()
            frames.append(df)
    merra = pd.concat(frames, ignore_index=True)
    merra = merra.groupby("Date", as_index=False).mean()
    return merra

merra_df = extract_merra_features_xr("merra_downloads")

# -------------------- STEP 4: MERGE ALL --------------------
combined_df = (aod_df.merge(pm_df, on="Date", how="inner")
                        .merge(merra_df, on="Date", how="inner"))

# -------------------- STEP 5: CLEAN & FEATURE ENGINEERING --------------------
combined_df["PM2.5 (µg/m³)"] = pd.to_numeric(combined_df["PM2.5 (µg/m³)"], errors="coerce")

features = ["Mean_AOD"] + MERRA_VARS
clean_df = combined_df.dropna(subset=features + ["PM2.5 (µg/m³)"]).copy()

clean_df["Temp_Diff"] = clean_df["TS"] - clean_df["T2M"]
clean_df["Humidity_Ratio"] = clean_df["QV2M"] / (clean_df["T2M"] + 1e-3)

X_cols = features + ["Temp_Diff", "Humidity_Ratio"]

# -------------------- STEP 6: TIME-AWARE TRAIN/TEST SPLIT --------------------
clean_df = clean_df.sort_values("Date")
split_idx = int(len(clean_df) * 0.8)
train_df = clean_df.iloc[:split_idx]
test_df = clean_df.iloc[split_idx:]

X_train, y_train = train_df[X_cols], train_df["PM2.5 (µg/m³)"]
X_test, y_test = test_df[X_cols], test_df["PM2.5 (µg/m³)"]

# -------------------- STEP 7: TRAIN MODEL --------------------
rf = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# -------------------- STEP 8: EVALUATE --------------------
y_pred = rf.predict(X_test)
print("✅ Evaluation Metrics:")
print("MAE :", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R²  :", r2_score(y_test, y_pred))

# -------------------- STEP 9: PREDICT FOR A NEW DAY --------------------
def extract_merra_day_mean(file_path, date_ist, lat=CITY_LAT, lon=CITY_LON):
    ds = xr.open_dataset(file_path)
    sub = ds.sel(lat=lat, lon=lon, method="nearest")[MERRA_VARS]
    df = sub.to_dataframe().reset_index()
    df["Date"] = (pd.to_datetime(df["time"])
                  .dt.tz_localize("UTC")
                  .dt.tz_convert(IST)
                  .dt.floor("D")
                  .dt.tz_localize(None))
    day = pd.to_datetime(date_ist)
    day_df = df[df["Date"] == day]
    feats = day_df[MERRA_VARS].mean().to_dict()
    return feats

# Example date: 2024-05-10
merra_file = "merra_downloads/MERRA2_400.tavg1_2d_slv_Nx.20240511.SUB.nc"
merra_features = extract_merra_day_mean(merra_file, "2024-05-11")
merra_features["Mean_AOD"] = 0.97  # Replace with real AOD if available

predict_input_df = pd.DataFrame([merra_features])
predict_input_df["Temp_Diff"] = predict_input_df["TS"] - predict_input_df["T2M"]
predict_input_df["Humidity_Ratio"] = predict_input_df["QV2M"] / (predict_input_df["T2M"] + 1e-3)

pred_pm = rf.predict(predict_input_df[X_cols])[0]
print(f"\n🔮 Predicted PM2.5 for 2024-05-11 IST: {pred_pm}")

# -------------------- STEP 10: SAVE PREDICTION --------------------
output_df = pd.DataFrame({
    "Date": ["2024-05-10"],
    "Predicted_PM2.5": [pred_pm]
})
output_df.to_csv("predicted_pm25.csv", index=False)
print("💾 Prediction saved to 'predicted_pm25.csv'")


✅ Evaluation Metrics:
MAE : 153.92415000000034
RMSE: 160.2998252595538
R²  : -13.265200219749412

🔮 Predicted PM2.5 for 2024-05-11 IST: 122.24639999999918
💾 Prediction saved to 'predicted_pm25.csv'


In [13]:
#Code including the approach of clustering as well as regression
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.cluster import KMeans
from netCDF4 import Dataset, num2date
from datetime import datetime
import os
import xarray as xr
import re

def extract_aod_from_nc(nc_folder, lat=28.41, lon=77.31):
    """
    Read .nc files in `nc_folder`, detect coordinate/var names automatically,
    extract AOD at nearest gridpoint to (lat, lon) for each time step or snapshot.
    Returns DataFrame with columns: Date, Mean_AOD_nc
    """
    records = []
    for fname in os.listdir(nc_folder):
        if not fname.endswith(".nc"):
            continue
        path = os.path.join(nc_folder, fname)
        ds = xr.open_dataset(path, decode_times=False)  # decode_times False initially

        # --- detect time coordinate name ---
        time_candidates = ['time', 'Time', 'TIME', 'times', 'date', 'Date', 'datetime', 'DATETIME', 't', 'TIME_UTC']
        time_name = next((c for c in time_candidates if c in ds.coords or c in ds.variables), None)

        if time_name is None:
            # try automatic detection by dtype (variables that are datetime64)
            for k in ds.variables:
                try:
                    if np.issubdtype(ds[k].values.dtype, np.datetime64):
                        time_name = k
                        break
                except Exception:
                    pass
        if time_name is None:
            # try xarray's decoded time via decode_cf
            try:
                ds2 = xr.open_dataset(path, decode_times=True)
                time_name = next((c for c in ds2.coords if np.issubdtype(ds2[c].values.dtype, np.datetime64)), None)
                if time_name:
                    ds = ds2
            except Exception:
                pass

        # ----------- UPDATED SECTION: handle no time coordinate --------------
        if time_name is None:
            match = re.search(r'(\d{8})', fname)
            if match:
                date_val = pd.to_datetime(match.group(1), format='%Y%m%d')
            else:
                date_val = pd.Timestamp.today().normalize()
                print(f"⚠️ Could not find date in filename {fname}, using today's date.")

            lat_candidates = ['lat', 'latitude', 'LAT', 'Latitude']
            lon_candidates = ['lon', 'longitude', 'LON', 'Longitude']
            lat_name = next((c for c in lat_candidates if c in ds.coords or c in ds.variables), None)
            lon_name = next((c for c in lon_candidates if c in ds.coords or c in ds.variables), None)
            if lat_name is None or lon_name is None:
                raise ValueError(f"No lat/lon coords found in {fname}. coords: {list(ds.coords.keys())}")
            aod_var = next((v for v in ds.data_vars if 'aod' in v.lower()), None)
            if aod_var is None:
                raise ValueError(f"No AOD-like variable found in {fname}. data_vars: {list(ds.data_vars)}")
            lat_vals = ds[lat_name].values
            lon_vals = ds[lon_name].values
            aod_data = ds[aod_var].values

            if getattr(lat_vals, 'ndim', 1) == 1 and getattr(lon_vals, 'ndim', 1) == 1:
                nearest_lat = ds[lat_name].sel({lat_name: lat}, method="nearest").values
                nearest_lon = ds[lon_name].sel({lon_name: lon}, method="nearest").values
                aod_value = float(ds[aod_var].sel({lat_name: nearest_lat, lon_name: nearest_lon}).values)
                records.append({"Date": date_val, "Mean_AOD_nc": aod_value})
            else:
                flat_lat = lat_vals.ravel()
                flat_lon = lon_vals.ravel()
                dist = (flat_lat - lat)**2 + (flat_lon - lon)**2
                flat_idx = np.argmin(dist)
                ij = np.unravel_index(flat_idx, lat_vals.shape)
                aod_value = float(aod_data[ij])
                records.append({"Date": date_val, "Mean_AOD_nc": aod_value})
            continue

        # --- detect lat/lon names ---
        lat_candidates = ['lat', 'latitude', 'LAT', 'Latitude']
        lon_candidates = ['lon', 'longitude', 'LON', 'Longitude']
        lat_name = next((c for c in lat_candidates if c in ds.coords or c in ds.variables), None)
        lon_name = next((c for c in lon_candidates if c in ds.coords or c in ds.variables), None)
        if lat_name is None or lon_name is None:
            raise ValueError(f"No lat/lon coords found in {fname}. coords: {list(ds.coords.keys())}")
        aod_var = next((v for v in ds.data_vars if 'aod' in v.lower()), None)
        if aod_var is None:
            raise ValueError(f"No AOD-like variable found in {fname}. data_vars: {list(ds.data_vars)}")
        lat_vals = ds[lat_name].values
        lon_vals = ds[lon_name].values

        if getattr(lat_vals, 'ndim', 1) == 1 and getattr(lon_vals, 'ndim', 1) == 1:
            nearest_lat = ds[lat_name].sel({lat_name: lat}, method="nearest").values
            nearest_lon = ds[lon_name].sel({lon_name: lon}, method="nearest").values
            time_vals = ds[time_name].values
            for t in time_vals:
                try:
                    date_val = pd.to_datetime(str(t)).normalize()
                except Exception:
                    date_val = pd.to_datetime(t).normalize()
                aod_value = float(ds[aod_var].sel({time_name: t, lat_name: nearest_lat, lon_name: nearest_lon}).values)
                records.append({"Date": date_val, "Mean_AOD_nc": aod_value})
        else:
            lat_arr = np.array(lat_vals)
            lon_arr = np.array(lon_vals)
            if lat_arr.shape != lon_arr.shape:
                raise ValueError(f"Lat/Lon shapes differ in {fname}: {lat_arr.shape} vs {lon_arr.shape}")
            flat_lat = lat_arr.ravel()
            flat_lon = lon_arr.ravel()
            dist = (flat_lat - lat)**2 + (flat_lon - lon)**2
            flat_idx = np.argmin(dist)
            ij = np.unravel_index(flat_idx, lat_arr.shape)
            aod_da = ds[aod_var]
            lat_dim = next((d for d in aod_da.dims if d in ds[lat_name].dims or 'lat' in d.lower()), None)
            lon_dim = next((d for d in aod_da.dims if d in ds[lon_name].dims or 'lon' in d.lower()), None)
            if lat_dim is None or lon_dim is None:
                lat_dim = aod_da.dims[1]
                lon_dim = aod_da.dims[2]
            for t in ds[time_name].values:
                try:
                    date_val = pd.to_datetime(str(t)).normalize()
                except Exception:
                    date_val = pd.to_datetime(t).normalize()
                aod_slice = aod_da.sel({time_name: t})
                aod_value = float(aod_slice.isel({lat_dim: ij[0], lon_dim: ij[1]}).values)
                records.append({"Date": date_val, "Mean_AOD_nc": aod_value})
    return pd.DataFrame(records)

def extract_merra_features(nc_folder):
    records = []
    for file in os.listdir(nc_folder):
        if file.endswith(".nc"):
            ds = Dataset(os.path.join(nc_folder, file), 'r')
            time_var = ds.variables['time']
            times = num2date(time_var[:], units=time_var.units, only_use_cftime_datetimes=False)
            ps = ds.variables['PS'][:, 0, 0]
            qv2m = ds.variables['QV2M'][:, 0, 0]
            t2m = ds.variables['T2M'][:, 0, 0]
            ts = ds.variables['TS'][:, 0, 0]
            u10m = ds.variables['U10M'][:, 0, 0]
            qv10m = ds.variables['QV10M'][:, 0, 0]
            slp = ds.variables['SLP'][:, 0, 0]
            t10m = ds.variables['T10M'][:, 0, 0]
            t2mdew = ds.variables['T2MDEW'][:, 0, 0]
            tqi = ds.variables['TQI'][:, 0, 0]
            tql = ds.variables['TQL'][:, 0, 0]
            for i in range(len(times)):
                date_val = times[i]
                if hasattr(date_val, 'year'):
                    date_val = datetime(date_val.year, date_val.month, date_val.day)
                records.append({
                    "Date": date_val,
                    "PS": ps[i],
                    "QV2M": qv2m[i],
                    "T2M": t2m[i],
                    "TS": ts[i],
                    "U10M": u10m[i],
                    "QV10M": qv10m[i],
                    "SLP": slp[i],
                    "T10M": t10m[i],
                    "T2MDEW": t2mdew[i],
                    "TQI": tqi[i],
                    "TQL": tql[i],
                })
    return pd.DataFrame(records)

def extract_merra_single_day(file_path, lat=28.41, lon=77.31):
    ds = xr.open_dataset(file_path)
    nearest_lat = ds.sel(lat=lat, method="nearest").lat.values
    nearest_lon = ds.sel(lon=lon, method="nearest").lon.values
    time_step = ds.time.values[0]  # Assuming only one day present
    features = {
        "PS":     [float(ds["PS"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "QV2M":   [float(ds["QV2M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "T2M":    [float(ds["T2M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "TS":     [float(ds["TS"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "U10M":   [float(ds["U10M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "QV10M":  [float(ds["QV10M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "SLP":    [float(ds["SLP"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "T10M":   [float(ds["T10M"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "T2MDEW": [float(ds["T2MDEW"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "TQI":    [float(ds["TQI"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
        "TQL":    [float(ds["TQL"].sel(time=time_step, lat=nearest_lat, lon=nearest_lon).values)],
    }
    return features

# -------------------- DATA LOADING --------------------

aod_nc = extract_aod_from_nc("Latest_Data")
aod_nc['Date'] = pd.to_datetime(aod_nc['Date'])
aod_nc.rename(columns={'Mean_AOD_nc': 'Mean_AOD'}, inplace=True)  # Ensure unified name

aod_csv = pd.read_csv("aod_data.csv")
aod_csv['Date'] = pd.to_datetime(aod_csv['Date'])

pm_df = pd.read_csv("pm25_cpcb_05AM_01janto18june.csv")
pm_df['Timestamp'] = pd.to_datetime(pm_df['Timestamp'])
pm_df['Date'] = pm_df['Timestamp'].dt.normalize()

merra_df = extract_merra_features("merra_downloads")
merra_df = merra_df.groupby("Date").mean().reset_index()
merra_df['Date'] = pd.to_datetime(merra_df['Date'])

# Merge all on Date
combined_df = pd.merge(aod_csv, pm_df, on="Date", how="outer")
combined_df = pd.merge(combined_df, aod_nc, on="Date", how="outer")
combined_df = pd.merge(combined_df, merra_df, on="Date", how="outer")

# ------------------- ML & CLUSTERING -------------------
features = ['Mean_AOD', 'PS', 'QV2M', 'T2M', 'TS', 'U10M', 'QV10M', 'SLP', 'T10M', 'T2MDEW', 'TQI', 'TQL']
clean_df = combined_df.dropna(subset=features + ['PM2.5 (µg/m³)']).copy()
clean_df['Temp_Diff'] = clean_df['TS'] - clean_df['T2M']
clean_df['Humidity_Ratio'] = clean_df['QV2M'] / (clean_df['T2M'] + 1e-3)

# ---------- CLUSTERING BY PM2.5 ----------
num_clusters = 2   # you can choose more if you want
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clean_df['Cluster'] = kmeans.fit_predict(clean_df['PM2.5 (µg/m³)'].values.reshape(-1, 1))

# ---------- PER-CLUSTER REGRESSION ----------
cluster_models = {}

for cluster_label in range(num_clusters):
    cluster_data = clean_df[clean_df['Cluster'] == cluster_label]
    # Features for model
    X_clust = cluster_data[features].copy()
    X_clust['Temp_Diff'] = cluster_data['Temp_Diff']
    X_clust['Humidity_Ratio'] = cluster_data['Humidity_Ratio']
    y_clust = cluster_data['PM2.5 (µg/m³)']

    # Split and train
    X_train, X_test, y_train, y_test = train_test_split(X_clust, y_clust, test_size=0.2, random_state=42)
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    cluster_models[cluster_label] = rf

    # Evaluation
    y_pred = rf.predict(X_test)
    print(f"\n✅ Cluster {cluster_label} Evaluation:")
    print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
    print(f"R²: {r2_score(y_test, y_pred):.3f}")

# ---------------- PREDICTION FOR NEW DAY -------------------
# Supply values for target day
merra_file = "merra_downloads/MERRA2_400.tavg1_2d_slv_Nx.20240124.SUB.nc"  # <== Change for desired date
merra_features = extract_merra_single_day(merra_file)
merra_features["Mean_AOD"] = [0.90]   # <== Change for desired date, or fetch from source

predict_input_df = pd.DataFrame(merra_features)
predict_input_df['Temp_Diff'] = predict_input_df['TS'] - predict_input_df['T2M']
predict_input_df['Humidity_Ratio'] = predict_input_df['QV2M'] / (predict_input_df['T2M'] + 1e-3)
X_input = predict_input_df[features].copy()
X_input['Temp_Diff'] = predict_input_df['Temp_Diff']
X_input['Humidity_Ratio'] = predict_input_df['Humidity_Ratio']

# CLUSTER ASSIGNMENT for the new day
# Option 1: Assign based on nearest cluster center by feature proxy (Mean_AOD, etc if available)
# Option 2: Assign using mean PM2.5 value in clusters (if you expect pollution level) OR use cluster 0 or 1

# Here we assign cluster with kmeans using a feature proxy. Since our clustering was on PM2.5,
# and we don't have true PM2.5 for the day to cluster, just use both models and compare OR pick one.

predictions = {}
for cluster_label, model in cluster_models.items():
    pred = model.predict(X_input)[0]
    predictions[cluster_label] = pred

print("\n🔮 Cluster-wise Predictions for new day:")
for cluster_label, pred in predictions.items():
    print(f"Cluster {cluster_label}: Predicted PM2.5 = {pred:.2f}")

# You can pick the prediction you want, e.g., highest, lowest, or based on expected pollution scenario
# Here let's pick the highest
best_cluster = max(predictions, key=predictions.get)
final_prediction = predictions[best_cluster]

# --- extract date from filename (for output) ---
match = re.search(r'(\d{8})', merra_file)
if match:
    pred_date = pd.to_datetime(match.group(1), format='%Y%m%d').strftime('%Y-%m-%d')
else:
    pred_date = pd.Timestamp.today().strftime('%Y-%m-%d')

# ----- Save prediction -----
output_df = pd.DataFrame({
    'Date': [pred_date],
    'Predicted_PM2.5': [final_prediction],
    'Cluster_Used': [best_cluster]
})
output_df.to_csv('predicted_pm25.csv', index=False, mode='w')
print(f"\n💾 Prediction saved to 'predicted_pm25.csv' for {pred_date}, cluster {best_cluster}")


KeyError: ['Mean_AOD']